In [1]:
!pip show openai | grep Version

Version: 1.3.3


In [2]:
import json

def show_json(obj):
    display(json.loads(obj.model_dump_json()))

In [1]:
from openai import OpenAI

client = OpenAI()

In [18]:
assistant = client.beta.assistants.create(
    name="Math Tutor",
    instructions="You are a personal math tutor. Answer questions briefly, in a sentence or less.",
    model="gpt-4-1106-preview",
)
show_json(assistant)

{'id': 'asst_rwVCFXJKcE8CeDRdAe5er7u2',
 'created_at': 1700354434,
 'description': None,
 'file_ids': [],
 'instructions': 'You are a personal math tutor. Answer questions briefly, in a sentence or less.',
 'metadata': {},
 'model': 'gpt-4-1106-preview',
 'name': 'Math Tutor',
 'object': 'assistant',
 'tools': []}

In [7]:
thread = client.beta.threads.create()
show_json(thread)

{'id': 'thread_D3rtnzro7Z85K6Fz2oewh7gi',
 'created_at': 1700352964,
 'metadata': {},
 'object': 'thread'}

In [17]:
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="I need to solve the equation `3x + 11 = 14`. Can you help me?",
)
show_json(message)

{'id': 'msg_OkdfpdhTO9E89oNN0jd6cRsO',
 'assistant_id': None,
 'content': [{'text': {'annotations': [],
    'value': 'I need to solve the equation `3x + 11 = 14`. Can you help me?'},
   'type': 'text'}],
 'created_at': 1700354373,
 'file_ids': [],
 'metadata': {},
 'object': 'thread.message',
 'role': 'user',
 'run_id': None,
 'thread_id': 'thread_D3rtnzro7Z85K6Fz2oewh7gi'}

In [19]:
run = client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant.id,
)
show_json(run)

{'id': 'run_8kDLGAtm9M4i06vtnGclxXh9',
 'assistant_id': 'asst_rwVCFXJKcE8CeDRdAe5er7u2',
 'cancelled_at': None,
 'completed_at': None,
 'created_at': 1700354447,
 'expires_at': 1700355047,
 'failed_at': None,
 'file_ids': [],
 'instructions': 'You are a personal math tutor. Answer questions briefly, in a sentence or less.',
 'last_error': None,
 'metadata': {},
 'model': 'gpt-4-1106-preview',
 'object': 'thread.run',
 'required_action': None,
 'started_at': None,
 'status': 'queued',
 'thread_id': 'thread_D3rtnzro7Z85K6Fz2oewh7gi',
 'tools': []}

In [20]:
import time

def wait_on_run(run, thread):
    while run.status == "queued" or run.status == "in_progress":
        run = client.beta.threads.runs.retrieve(
            thread_id=thread.id,
            run_id=run.id,
        )
        time.sleep(0.5)
    return run

In [21]:
run = wait_on_run(run, thread)
show_json(run)

{'id': 'run_8kDLGAtm9M4i06vtnGclxXh9',
 'assistant_id': 'asst_rwVCFXJKcE8CeDRdAe5er7u2',
 'cancelled_at': None,
 'completed_at': 1700354449,
 'created_at': 1700354447,
 'expires_at': None,
 'failed_at': None,
 'file_ids': [],
 'instructions': 'You are a personal math tutor. Answer questions briefly, in a sentence or less.',
 'last_error': None,
 'metadata': {},
 'model': 'gpt-4-1106-preview',
 'object': 'thread.run',
 'required_action': None,
 'started_at': 1700354447,
 'status': 'completed',
 'thread_id': 'thread_D3rtnzro7Z85K6Fz2oewh7gi',
 'tools': []}

In [22]:
messages = client.beta.threads.messages.list(thread_id=thread.id)
show_json(messages)

{'data': [{'id': 'msg_SEgRy91lmMZuLVfR8FTtnENe',
   'assistant_id': 'asst_rwVCFXJKcE8CeDRdAe5er7u2',
   'content': [{'text': {'annotations': [],
      'value': 'Sure, subtract 11 from both sides to get `3x = 3`, then divide both sides by 3 to find `x = 1`.'},
     'type': 'text'}],
   'created_at': 1700354449,
   'file_ids': [],
   'metadata': {},
   'object': 'thread.message',
   'role': 'assistant',
   'run_id': 'run_8kDLGAtm9M4i06vtnGclxXh9',
   'thread_id': 'thread_D3rtnzro7Z85K6Fz2oewh7gi'},
  {'id': 'msg_OkdfpdhTO9E89oNN0jd6cRsO',
   'assistant_id': None,
   'content': [{'text': {'annotations': [],
      'value': 'I need to solve the equation `3x + 11 = 14`. Can you help me?'},
     'type': 'text'}],
   'created_at': 1700354373,
   'file_ids': [],
   'metadata': {},
   'object': 'thread.message',
   'role': 'user',
   'run_id': None,
   'thread_id': 'thread_D3rtnzro7Z85K6Fz2oewh7gi'}],
 'object': 'list',
 'first_id': 'msg_SEgRy91lmMZuLVfR8FTtnENe',
 'last_id': 'msg_OkdfpdhTO9E89o

In [23]:
# Create a message to append to our thread
message = client.beta.threads.messages.create(
    thread_id=thread.id, role="user", content="Could you explain this to me?"
)

# Execute our run
run = client.beta.threads.runs.create(
    thread_id=thread.id,
    assistant_id=assistant.id,
)

# Wait for completion
wait_on_run(run, thread)

# Retrieve all the messages added after our last user message
messages = client.beta.threads.messages.list(
    thread_id=thread.id, order="asc", after=message.id
)
show_json(messages)

{'data': [{'id': 'msg_8rZ67KzWnm4fvwHSWbMV2O0m',
   'assistant_id': 'asst_rwVCFXJKcE8CeDRdAe5er7u2',
   'content': [{'text': {'annotations': [],
      'value': 'Yes, to solve `3x + 11 = 14`, you want to isolate `x`. So you first subtract 11 from both sides to cancel the +11 on the left, leaving you with `3x = 3`. Then, you need to get x by itself, so you divide both sides of the equation by 3, which leaves you with `x = 1`. This is your solution.'},
     'type': 'text'}],
   'created_at': 1700355880,
   'file_ids': [],
   'metadata': {},
   'object': 'thread.message',
   'role': 'assistant',
   'run_id': 'run_wyLn50DqB27NzAqpVrO1c47g',
   'thread_id': 'thread_D3rtnzro7Z85K6Fz2oewh7gi'}],
 'object': 'list',
 'first_id': 'msg_8rZ67KzWnm4fvwHSWbMV2O0m',
 'last_id': 'msg_8rZ67KzWnm4fvwHSWbMV2O0m',
 'has_more': False}

In [ ]:
import openai

openai.api_key = 'your-api-key'

prompt_text = """
I have a database query plan with the following steps. Please explain each step in one simple sentence:

1. SCAN c
2. SEARCH i (Covering Index IFK_InvoiceCustomerId, CustomerId=?)
3. SEARCH ii (Index IFK_InvoiceLineInvoiceId, InvoiceId=?)
4. SEARCH t (Integer Primary Key, rowid=?)
5. SEARCH g (Integer Primary Key, rowid=?)
6. USE TEMP B-TREE (Group By)
7. USE TEMP B-TREE (Order By)
"""

response = openai.Completion.create(
    model="gpt-4-1106-preview",
    prompt=prompt_text,
    max_tokens=300
)

print(response.choices[0].text)
